# <center> <img src="../../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
### <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
**Primer Examen**

**Fecha**: 14 de Marzo del 2025

**Nombre del estudiante**: Angel Gabriel Ramirez Carrillo

**Professor**: Pablo Camarillo Ramirez

30bf90431f0101f98f7c61f1ad1cfa9c13515bc8dcf9dbdc

30bf90431f0101f98f7c61f1ad1cfa9c13515bc8dcf9dbdc

In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQL-Exam-1-Angel-Gabriel-Ramirez-Carrillo") \
    .master("spark://56a250e0d184:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

In [23]:
from pyspark.sql.functions import get_json_object
from pyspark.sql.functions import udf
from pyspark.sql.functions import col, when, avg, get_json_object, year, current_date
from pyspark.sql.types import (
    StructType, StructField, StringType, IntegerType, FloatType, DoubleType, BooleanType, 
    NullType, ShortType, LongType, ByteType, BinaryType, DecimalType, DateType, TimestampType, 
    TimestampNTZType, DayTimeIntervalType, ArrayType, MapType
)

In [12]:
employees_df = spark \
                .read \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/employees.csv")

departments_df = spark \
                .read \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/departments.csv")

In [14]:

employees_df = employees_df.withColumn("employee", get_json_object(employees_df.employee_info, "$.employee"))
departments_df = spark.read.option("header", "true").csv("/home/jovyan/notebooks/data/departments.csv")


In [22]:
employees_df.show(5)
departments_df.show(5)

+-----------+--------------------+--------+---------+-------------+---------+----------+
|employee_id|       employee_info|employee|     name|department_id|   salary| hire_date|
+-----------+--------------------+--------+---------+-------------+---------+----------+
|          1|{'name': 'Caitlyn...|    NULL|  Caitlyn|          103|115959.78|2002-06-10|
|          2|{'name': 'Rachel'...|    NULL|   Rachel|          104|100820.16|2009-07-01|
|          3|{'name': 'Carrie'...|    NULL|   Carrie|          105|114421.44|1998-12-10|
|          4|{'name': 'Renee',...|    NULL|    Renee|          104| 54688.13|1995-03-17|
|          5|{'name': 'Gabriel...|    NULL|Gabriella|          109|106267.03|1995-02-09|
+-----------+--------------------+--------+---------+-------------+---------+----------+
only showing top 5 rows

+-------------+--------------------+---------+
|department_id|     department_name| location|
+-------------+--------------------+---------+
|          101|     Human Resourc

In [18]:
employees_df = employees_df.withColumn("name", get_json_object(col("employee_info"), "$.name")) \
                           .withColumn("department_id", get_json_object(col("employee_info"), "$.department_id").cast(IntegerType())) \
                           .withColumn("salary", get_json_object(col("employee_info"), "$.salary").cast(DoubleType())) \
                           .withColumn("hire_date", get_json_object(col("employee_info"), "$.hire_date").cast(DateType()))


In [21]:
employee_join_departments = employees_df.join(departments_df, "department_id", "left")

employee_join_departments.show(5)

+-------------+-----------+--------------------+--------+---------+---------+----------+-------------------+-------------+
|department_id|employee_id|       employee_info|employee|     name|   salary| hire_date|    department_name|     location|
+-------------+-----------+--------------------+--------+---------+---------+----------+-------------------+-------------+
|          103|          1|{'name': 'Caitlyn...|    NULL|  Caitlyn|115959.78|2002-06-10|Sales and Marketing|      Chicago|
|          104|          2|{'name': 'Rachel'...|    NULL|   Rachel|100820.16|2009-07-01|   Data Engineering|      Zapopan|
|          105|          3|{'name': 'Carrie'...|    NULL|   Carrie|114421.44|1998-12-10|       Data Science|      Seattle|
|          104|          4|{'name': 'Renee',...|    NULL|    Renee| 54688.13|1995-03-17|   Data Engineering|      Zapopan|
|          109|          5|{'name': 'Gabriel...|    NULL|Gabriella|106267.03|1995-02-09|   Customer Service|San Francisco|
+-------------+-

In [25]:
employee_join_departments = employee_join_departments.withColumn("salary_category", when(col("salary") >= 55000, "High").otherwise("Low"))
employee_join_departments.show(5)

+-------------+-----------+--------------------+--------+---------+---------+----------+-------------------+-------------+---------------+
|department_id|employee_id|       employee_info|employee|     name|   salary| hire_date|    department_name|     location|salary_category|
+-------------+-----------+--------------------+--------+---------+---------+----------+-------------------+-------------+---------------+
|          103|          1|{'name': 'Caitlyn...|    NULL|  Caitlyn|115959.78|2002-06-10|Sales and Marketing|      Chicago|           High|
|          104|          2|{'name': 'Rachel'...|    NULL|   Rachel|100820.16|2009-07-01|   Data Engineering|      Zapopan|           High|
|          105|          3|{'name': 'Carrie'...|    NULL|   Carrie|114421.44|1998-12-10|       Data Science|      Seattle|           High|
|          104|          4|{'name': 'Renee',...|    NULL|    Renee| 54688.13|1995-03-17|   Data Engineering|      Zapopan|            Low|
|          109|          5|

In [27]:
high_salary_df = employee_join_departments.filter(col("salary_category") == "High")
low_salary_df = employee_join_departments.filter(col("salary_category") == "Low")

high_salary_df.show(5)
low_salary_df.show(5)

+-------------+-----------+--------------------+--------+---------+---------+----------+-------------------+-------------+---------------+
|department_id|employee_id|       employee_info|employee|     name|   salary| hire_date|    department_name|     location|salary_category|
+-------------+-----------+--------------------+--------+---------+---------+----------+-------------------+-------------+---------------+
|          103|          1|{'name': 'Caitlyn...|    NULL|  Caitlyn|115959.78|2002-06-10|Sales and Marketing|      Chicago|           High|
|          104|          2|{'name': 'Rachel'...|    NULL|   Rachel|100820.16|2009-07-01|   Data Engineering|      Zapopan|           High|
|          105|          3|{'name': 'Carrie'...|    NULL|   Carrie|114421.44|1998-12-10|       Data Science|      Seattle|           High|
|          109|          5|{'name': 'Gabriel...|    NULL|Gabriella|106267.03|1995-02-09|   Customer Service|San Francisco|           High|
|          101|          6|

In [29]:
high_avg_salary = high_salary_df.groupBy("department_name").agg(avg("salary").alias("avg_salary"))
low_avg_salary = low_salary_df.groupBy("department_name").agg(avg("salary").alias("avg_salary"))

high_avg_salary.show(5)
low_avg_salary.show(5)

+--------------------+------------------+
|     department_name|        avg_salary|
+--------------------+------------------+
|Corporate Strateg...|102741.38324414717|
| Sales and Marketing|100839.65275449108|
|    Data Engineering|101626.29492163012|
|Research and Deve...|  98714.3003086419|
|Finance and Accou...|100731.07877887784|
+--------------------+------------------+
only showing top 5 rows

+--------------------+------------------+
|     department_name|        avg_salary|
+--------------------+------------------+
|Corporate Strateg...|41590.741833333326|
| Sales and Marketing| 41150.40277777778|
|    Data Engineering| 41358.50794117647|
|Finance and Accou...|42740.952888888874|
|Research and Deve...| 41426.43521126761|
+--------------------+------------------+
only showing top 5 rows



In [31]:
top_5_high_salary = high_salary_df.orderBy(col("salary").desc()).limit(5)
top_5_low_salary = low_salary_df.orderBy(col("salary").desc()).limit(5)

top_5_high_salary.show(5)
top_5_low_salary.show(5)

+-------------+-----------+--------------------+--------+---------+---------+----------+--------------------+-----------+---------------+
|department_id|employee_id|       employee_info|employee|     name|   salary| hire_date|     department_name|   location|salary_category|
+-------------+-----------+--------------------+--------+---------+---------+----------+--------------------+-----------+---------------+
|          101|       1778|{'name': 'Gabriel...|    NULL|Gabriella|149989.73|2018-09-14|     Human Resources|  San Diego|           High|
|          101|       3621|{'name': 'Katheri...|    NULL|Katherine| 149979.3|2017-07-26|     Human Resources|  San Diego|           High|
|          110|        346|{'name': 'Ryan', ...|    NULL|     Ryan| 149963.1|1990-07-03|Corporate Strateg...|Los Angeles|           High|
|          107|       3807|{'name': 'Caitlyn...|    NULL|  Caitlyn|149956.54|2000-07-27|               Legal|    Chicago|           High|
|          107|       3050|{'name'

In [38]:
employee_join_departments = employee_join_departments.withColumn("years_in_company", year(current_date()) - year(col("hire_date")))
max_years = employee_join_departments.agg({"years_in_company": "max"}).collect()[0][0]
employees_with_max_years = employee_join_departments.filter(col("years_in_company") == max_years)
employees_count = employees_with_max_years.count()
print(employees_count)



88


In [ ]:
sc.stop()